In [1]:
%pip install -qU langchain langchain_openai langsmith pandas langchain_experimental langgraph langchain_core

Note: you may need to restart the kernel to use updated packages.


In [2]:
# api keys
import getpass
import os

# Optional, add tracing in LangSmith


# Create Agent

The `create_agent` function is used to create an AI agent with a specific language model and a set of tools. The function takes three arguments: `llm`, `tools`, and `system_message`.

The `llm` argument is the language model that will be used by the agent. This model is responsible for generating the agent's responses.

The `tools` argument is a list of tools that the agent has access to. These tools are transformed into functions that the language model can use, by using a list comprehension to apply the `format_tool_to_openai_function` function to each tool.

The `system_message` argument is a string that will be included in the prompt that the agent uses to generate its responses.

The function first creates a `ChatPromptTemplate` object, which is a template for the prompts that the agent will use. This template includes a system message that describes the agent's role and instructions, as well as a placeholder for additional messages.

The `system_message` and `tool_names` are then added to the prompt using the `partial` method, which fills in parts of the template.

Finally, the function binds the functions derived from the tools to the language model and combines this with the prompt using the `|` operator. This creates a pipeline where the prompt is used to generate a response from the language model, and the functions are used to process the response. The resulting agent is then returned by the function.

In [3]:
import json

from langchain_core.messages import (
    AIMessage,
    BaseMessage,
    ChatMessage,
    FunctionMessage,
    HumanMessage,
)
from langchain.tools.render import format_tool_to_openai_function
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import END, StateGraph
from langgraph.prebuilt.tool_executor import ToolExecutor, ToolInvocation

def create_agent(llm, tools, system_message: str):
    """
    Create an agent.

    Args:
        llm: The language model to bind the functions to.
        tools: The list of tools available to the agent.
        system_message: The system message to be included in the prompt.

    Returns:
        The created agent.

    """
    functions = [format_tool_to_openai_function(t) for t in tools] # format the tools to OpenAI functions 

    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are a helpful AI assistant, collaborating with other assistants."
                " Use the provided tools to progress towards answering the question."
                " If you are unable to fully answer, that's OK, another assistant with different tools "
                " will help where you left off. Execute what you can to make progress."
                " If you or any of the other assistants have the final answer or deliverable,"
                " prefix your response with FINAL ANSWER so the team knows to stop."
                " You have access to the following tools: {tool_names}.\n{system_message}",
            ),
            MessagesPlaceholder(variable_name="messages"),
        ]
    )
    
    prompt = prompt.partial(system_message=system_message) # add the system message to the prompt
    prompt = prompt.partial(tool_names=", ".join([tool.name for tool in tools])) # add the tool names to the prompt
    
    return prompt | llm.bind_functions(functions) # output of the prompt is used as the input for the llm with its bound functions

In [4]:
from langchain_core.tools import tool
from typing import Annotated
from langchain_experimental.utilities import PythonREPL
from langchain_community.tools.tavily_search import TavilySearchResults

tavily_tool = TavilySearchResults(max_results=5) # create a Tavily search tool to search for results

# Warning: This executes code locally, which can be unsafe when not sandboxed
repl = PythonREPL() # create a Python REPL tool to execute Python code


@tool # add the tool decorator to the function
def python_repl(
    code: Annotated[str, "The python code to execute in the REPL."]
):
    """
    Use this function to execute Python code.
    
    Args:
        code (str): The Python code to execute.
        
    Returns:
        str: A string indicating the result of the execution and the stdout.
    """
    try:
        result = repl.run(code)
    except BaseException as e:
        return f"Failed to execute. Error: {repr(e)}"
    return f"Succesfully executed:\n```python\n{code}\n```\nStdout: {result}"

In [18]:
from IPython.display import Markdown

def print_md(md):
    display(Markdown(md))

print_md(python_repl("print('Hello, World!')")) # test the Python REPL tool

Succesfully executed:
```python
print('Hello, World!')
```
Stdout: Hello, World!
